In [ ]:
import pandas as pd
import os
from typing import List
import datetime

In [ ]:

# Specify the directory path as a string
directory_path: str = 'data/231102'

# List to store the names of CSV files (each file name is a string)
csv_files: List[str] = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# List to hold DataFrame objects
all_data: List[pd.DataFrame] = []

# Iterate over the list of file names
for file_name in csv_files:
    file_path: str = os.path.join(directory_path, file_name)  # Construct full file path
    df: pd.DataFrame = pd.read_csv(file_path)  # Read CSV into DataFrame
    all_data.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_data: pd.DataFrame = pd.concat(all_data)

In [ ]:
combined_data['time'] = pd.to_datetime(combined_data['time']) 

combined_data.head()

In [ ]:
specific_date = '2023-11-02'
start_time = '14:00:00'
end_time = '14:30:00'

filtered_data = combined_data[
    (combined_data['time'].dt.date == pd.to_datetime(specific_date).date()) &
    (combined_data['time'].dt.time >= pd.to_datetime(start_time).time()) &
    (combined_data['time'].dt.time <= pd.to_datetime(end_time).time())
]

filtered_data

In [ ]:
mac_counts = filtered_data.groupby('mac_id').size()

# Convert the Series to a dictionary (optional)
mac_counts_dict = mac_counts.to_dict()
mac_counts_dict.__len__()

In [ ]:
filtered_mac_counts_dict = {key: value for key, value in mac_counts_dict.items() if value >= 120}
filtered_mac_counts_dict.__len__()

In [ ]:
mac_addresses_to_keep = list(filtered_mac_counts_dict.keys())

filtered_data = filtered_data[filtered_data['mac_id'].isin(mac_addresses_to_keep)]

In [ ]:
grouped = filtered_data.groupby('mac_id')['zone_id'].apply(set)

# Convert to dictionary
mac_id_to_zone_id_dict = grouped.to_dict()

mac_id_to_zone_id_dict.__len__()

In [ ]:
filtered_mac_id_to_zone_id_dict = {
    mac_id: zone_ids
    for mac_id, zone_ids in mac_id_to_zone_id_dict.items()
    if len(zone_ids) >= 15
}
filtered_mac_id_to_zone_id_dict.__len__()

In [ ]:
mac_ids_to_keep = list(filtered_mac_id_to_zone_id_dict.keys())

filtered_data = filtered_data[filtered_data['mac_id'].isin(mac_ids_to_keep)]

filtered_data

In [ ]:
pivot_df = filtered_data.pivot_table(index='time', columns='mac_id', values='zone_id', aggfunc='first')
pivot_df = pivot_df.bfill().ffill()

In [ ]:
# Number of columns in pivot_df
num_columns = len(pivot_df.columns)

# Create a list of integers [1, 2, 3, ..., num_columns]
new_column_names = list(range(1, num_columns + 1))

# Assign the new column names to pivot_df
pivot_df.columns = new_column_names

pivot_df

In [ ]:
def mode_agg(series):
    if len(series) > 0:
        return series.mode()[0]
    else:
        return pd.NA

In [ ]:
resampled_df = pivot_df.resample('1T')

pivot_df = resampled_df.agg(mode_agg)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pivot_df[1]

In [ ]:
zone_position_map = {
        1: (45, 45),
        2: (45, 119),
        3: (45, 193),
        4: (45, 267),
        5: (45, 341),
        6: (141, 45),
        7: (141, 119),
        8: (141, 193),
        9: (141, 267),
        10: (141, 341),
        11: (237, 45),
        12: (237, 119),
        13: (237, 193),
        14: (237, 267),
        15: (237, 341),
        16: (333, 45),
        17: (333, 119),
        18: (333, 193),
        19: (333, 267),
        20: (333, 341),
        21: (429, 45),
        22: (429, 119),
        23: (429, 193),
        23: (429, 267),
        24: (429, 341),
        25: (525, 45),
    }

In [ ]:
transformed_df = pivot_df.map(lambda zone_id: zone_position_map.get(zone_id, (None, None)))
transformed_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Assuming 'mac_id' is the MAC address you want to visualize
mac_id = 2  # replace with the actual MAC address

# Extract the coordinates for the specific MAC address
mac_data = transformed_df[mac_id].dropna()  # Drop any NaNs

# Extract x and y coordinates
x_coords, y_coords = zip(*mac_data)
# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(x_coords, y_coords, marker='o')  # Use 'scatter' instead of 'plot' for a scatter plot

plt.title(f'Movement of MAC ID: {mac_id} Over Time')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.grid(True)
plt.show()

In [ ]:
mac_data

In [ ]:
x_coords, y_coords = zip(*mac_data)

y_coords

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation


# Assuming 'mac_id' is the MAC address you want to visualize
mac_id = 3  # replace with the actual MAC address

# Extract the coordinates for the specific MAC address
mac_data = transformed_df[mac_id].dropna()  # Drop any NaNs

# Extract x and y coordinates
x_coords, y_coords = zip(*mac_data)

# Noise parameters
mean = 0  # Mean of the noise
std_dev = 10  # Standard deviation of the noise

# Generate noise for x and y coordinates
noise_x = np.random.normal(mean, std_dev, len(x_coords))
noise_y = np.random.normal(mean, std_dev, len(y_coords))

# Add noise to the original coordinates
noisy_x_coords = [x + noise for x, noise in zip(x_coords, noise_x)]
noisy_y_coords = [y + noise for y, noise in zip(y_coords, noise_y)]

fig, ax = plt.subplots()
ax.set_xlim([0, 500])  # Adjust these limits based on your data range
ax.set_ylim([0, 500])

x, y = [], []

line, = ax.plot([], [], 'b.-')  # Initialize a line plot

def init():
    line.set_data([], [])
    return line,

def animate(frame):
    x.append(noisy_x_coords[frame])
    y.append(noisy_y_coords[frame])
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init, frames=len(x_coords), interval=1000, blit=True)

anim.save('animation.gif', writer='pillow')

plt.show()
